# HW3

In [1]:
import numpy as np
import pandas as pd 
data = pd.read_csv("AirbnbTrain.csv") 
test= pd.read_csv("AirbnbTest.csv") 
from gurobipy import *

In [2]:
x = pd.DataFrame(data)
test = pd.DataFrame(test) 
print(test.shape)
real=test["price"]
test.drop('price', axis=1,inplace=True)
print(test.shape)

y=data["price"]
x.head()
m=x.shape[1]-1
n=len(y)

#print(m)
#print(n)

(699, 13)
(699, 12)


## Q1

In [3]:
mod = Model()
#x = mod.addVars(n,m)# i->n j->m
#y = mod.addVars(n)
beta = mod.addVars(m)
z = mod.addVars(n)

for i in range(n):
    mod.addConstr(z[i]>=sum(beta[j]*x.iloc[i,j] for j in range(m))-y[i])
    mod.addConstr(z[i]>=y[i] - sum(beta[j]*x.iloc[i,j] for j in range(m)))   

# Construct objective
mod.setObjective(sum(z[i]/n for i in range(n)), GRB.MINIMIZE)
mod.update()
mod.optimize()
betas=[]


for j in range(m):
    betas.append(beta[j].x)



test=np.asarray(test)


prediction_error=sum(abs(real[i]-sum(betas[j]*test[i,j] for j in range (m))) for i in range(699))/699
print('The prediction error is',prediction_error)
    

Academic license - for non-commercial use only
Optimize a model with 3400 rows, 1712 columns and 41368 nonzeros
Coefficient statistics:
  Matrix range     [5e-01, 5e+02]
  Objective range  [6e-04, 6e-04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+03]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve time: 0.04s
Presolved: 3400 rows, 1712 columns, 41368 nonzeros

Ordering time: 0.00s

Barrier statistics:
 Dense cols : 12
 AA' NZ     : 3.967e+04
 Factor NZ  : 4.315e+04 (roughly 2 MBytes of memory)
 Factor Ops : 5.491e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.12302506e+03  0.00000000e+00  1.36e+03 0.00e+00  2.56e+01     0s
   1   2.05671988e+03 -1.60051438e+00  0.00e+00 5.43e-03  1.75e+00     0s
   2   1.14614521e+03  2.86828649e+00  0.00e+00 7.00e-05  2.34e-01     0s
   3   1.133045

The prediction error is 35.62719512642499

## Q2

In [7]:
mod = Model()
#x = mod.addVars(n,m)# i->n j->m
#y = mod.addVars(n)
beta = mod.addVars(n)
z = mod.addVars(n)
bi=mod.addVars(m,vtype = GRB.BINARY)

mod.addConstr(sum(bi[j] for j in range(m))<=3)
for j in range(m):
    mod.addConstr(beta[j]<=bi[j]*1000)
    mod.addConstr(-beta[j]<=bi[j]*1000)
    


for i in range(n):
    mod.addConstr(z[i]>=sum(beta[j]*x.iloc[i,j] for j in range(m))-y[i])
    mod.addConstr(z[i]>=y[i] - sum(beta[j]*x.iloc[i,j] for j in range(m)))


# Construct objective
mod.setObjective(sum(z[i]/n for i in range(n)), GRB.MINIMIZE)
mod.update()
mod.optimize()
name=x.columns
for j in range(m):
    if bi[j].x==1:
        print ('The coefficient for',name[j],'is',beta[j].x,'.')
betas=[]        
for j in range(m):
    betas.append(beta[j].x)



test=np.asarray(test)

prediction_error=sum(abs(real[i]-sum(betas[j]*test[i,j] for j in range (m))) for i in range(699))/699
print('The prediction error is',prediction_error)

Optimize a model with 3425 rows, 3412 columns and 41428 nonzeros
Variable types: 3400 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+03]
  Objective range  [6e-04, 6e-04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 2e+03]
Found heuristic solution: objective 144.9682353
Presolve removed 838 rows and 2101 columns
Presolve time: 0.05s
Presolved: 2587 rows, 1311 columns, 31294 nonzeros
Variable types: 1299 continuous, 12 integer (12 binary)

Root relaxation: objective 3.644463e+01, 1438 iterations, 0.29 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   36.44463    0    7  144.96824   36.44463  74.9%     -    0s
H    0     0                      61.6243748   36.44463  40.9%     -    0s
     0     0   36.44679    0   11   61.62437   36.44679  40.9%     -    0s
H    0     0                      41.0545511   36.4

### (a)

The coefficient for Entire home is 52.0 .
The coefficient for accommodates is 14.0 .
The coefficient for bedrooms is 32.0 .

### (b)

The prediction error is 37.73676680972818

## Q3

In [6]:
mod = Model()
#x = mod.addVars(n,m)# i->n j->m
#y = mod.addVars(n)
beta = mod.addVars(n)
z = mod.addVars(n)
bi=mod.addVars(m,vtype = GRB.BINARY)

mod.addConstr(sum(bi[j] for j in range(m))<=3)
mod.addConstr(bi[6]==1)
for j in range(m):
    mod.addConstr(beta[j]<=bi[j]*1000)
    mod.addConstr(-beta[j]<=bi[j]*1000)
    


for i in range(n):
    mod.addConstr(z[i]>=sum(beta[j]*x.iloc[i,j] for j in range(m))-y[i])
    mod.addConstr(z[i]>=y[i] - sum(beta[j]*x.iloc[i,j] for j in range(m)))


# Construct objective
mod.setObjective(sum(z[i]/n for i in range(n)), GRB.MINIMIZE)
mod.update()
mod.optimize()
name=x.columns
for j in range(m):
    if bi[j].x==1:
        print ('The coefficient for',name[j],'is',beta[j].x,'.')
betas=[]        
for j in range(m):
    betas.append(beta[j].x)



test=np.asarray(test)

prediction_error=sum(abs(real[i]-sum(betas[j]*test[i,j] for j in range (m))) for i in range(699))/699
print('The prediction error is',prediction_error)

Optimize a model with 3426 rows, 3412 columns and 41429 nonzeros
Variable types: 3400 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+03]
  Objective range  [6e-04, 6e-04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective 144.9682353
Presolve removed 840 rows and 2102 columns
Presolve time: 0.06s
Presolved: 2586 rows, 1310 columns, 31291 nonzeros
Variable types: 1299 continuous, 11 integer (11 binary)

Root relaxation: objective 3.644463e+01, 1449 iterations, 0.30 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   36.44463    0    8  144.96824   36.44463  74.9%     -    0s
H    0     0                      48.6742700   36.44463  25.1%     -    0s
     0     0   36.45800    0   10   48.67427   36.45800  25.1%     -    0s
H    0     0                      40.3043694   36.4

### (a)
The weight for Entire home is 67.875 .
The weight for bedrooms is 47.375 .
The weight for beds is 12.125 .



### (b)
The variable accomdates is dropped. This means that while accomodates is an important factor to consider, it isn't as significant as the bedrooms and entire home variables when it comes to calculating the least absolute deviations since it has the smallest coefficients among the three.

### (c)
The prediction error is 38.59960658082976